In [12]:
import torch
from torch.utils.data import Dataset
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

In [22]:
class TacDataset(Dataset):
    def __init__(self, path, sample_file, output_size, finger_type='both', variant=1):
        self.path = path
        self.output_size = output_size
        self.samples = np.loadtxt(Path(path) / sample_file).astype("int")

        if variant == 1:
            tact = torch.load(Path(path) / "all.pt")
            
        if finger_type == 'left':
            tact = tact[:,:39,...]
        elif finger_type == 'right':
            tact = tact[:,39:,...]
        
        self.tact = tact.reshape( tact.shape[0], -1, 1, 1, tact.shape[-1] )

    def __getitem__(self, index):
        input_index = self.samples[index, 0]
        class_label = self.samples[index, 1]
        target_class = torch.zeros((self.output_size, 1, 1, 1))
        target_class[class_label, ...] = 1

        return (
            self.tact[input_index],
            target_class,
            class_label,
        )

    def __len__(self):
        return self.samples.shape[0]

In [57]:
# train_dataset = ViTacDataset(
#     path=args.data_dir,
#     sample_file=f"train_{args.sample_file}.txt",
#     output_size=output_size,
#     mode=args.mode,
#     spiking=False,
#     rectangular=False,
# )

# train_loader = DataLoader(
#     dataset=train_dataset,
#     batch_size=args.batch_size,
#     shuffle=False,
#     num_workers=8,
# )
sample_file = 1
val_dataset = TacDataset(
    path='diagnostic_data/train_test_data/',
    sample_file=f"val_{sample_file}.txt",
    output_size=4,
    finger_type='left',
    variant=1
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8,
)

train_dataset = TacDataset(
    path='diagnostic_data/train_test_data/',
    sample_file=f"train_{sample_file}.txt",
    output_size=4,
    finger_type='left',
    variant=1
)
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8,
)

test_dataset = TacDataset(
    path='diagnostic_data/train_test_data/',
    sample_file=f"test.txt",
    output_size=4,
    finger_type='left',
    variant=1
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=8,
)

In [60]:
a,b,c = test_dataset[0]

In [61]:
a.shape, b.shape, c

(torch.Size([78, 1, 1, 110]), torch.Size([4, 1, 1, 1]), 0)

In [36]:
import torch
import slayerSNN as snn
from slayerSNN import loihi as spikeLayer

class SlayerMLP(torch.nn.Module):
    """2-layer MLP built using SLAYER's spiking layers."""

    def __init__(self, params, input_size, hidden_size, output_size):
        super(SlayerMLP, self).__init__()
        self.slayer = snn.layer(params["neuron"], params["simulation"])
        self.fc1 = self.slayer.dense(input_size, hidden_size)
        self.fc2 = self.slayer.dense(hidden_size, output_size)

    def forward(self, spike_input):
        spike_1 = self.slayer.spike(self.slayer.psp(self.fc1(spike_input)))
        spike_output = self.slayer.spike(self.slayer.psp(self.fc2(spike_1)))
        return spike_output

In [62]:
netParams = snn.params('network.yaml')
device = torch.device('cuda:02')

In [70]:
netParams.parameters

{'simulation': {'Ts': 1.0, 'tSample': 110},
 'neuron': {'type': 'SRMALPHA',
  'theta': 10,
  'tauSr': 20.0,
  'tauRef': 1.0,
  'scaleRef': 2,
  'tauRho': 1,
  'scaleRho': 1},
 'training': {'error': {'type': 'NumSpikes',
   'tgtSpikeRegion': {'start': 0, 'stop': 110},
   'tgtSpikeCount': {True: 30, False: 5}}}}

In [63]:
net = SlayerMLP(netParams, input_size=78, hidden_size=16, output_size=4).to(device)

In [64]:
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01, weight_decay=0.5)
error = snn.loss(netParams).to(device)

In [65]:
def _train():
    correct = 0
    num_samples = 0
    net.train()
    for data, target, label in train_loader:
        data = data.to(device)
        target = target.to(device)
        output = net.forward(data)
        correct += torch.sum(snn.predict.getClass(output) == label).data.item()
        num_samples += len(label)
        spike_loss = error.numSpikes(output, target)

        optimizer.zero_grad()
        spike_loss.backward()
        optimizer.step()
        
    print('Train:', correct / num_samples)

    #writer.add_scalar("loss/train", spike_loss / len(train_loader), epoch)
    #writer.add_scalar("acc/train", correct / num_samples, epoch)

In [66]:
def _val():
    correct = 0
    num_samples = 0
    net.eval()
    with torch.no_grad():
        for data, target, label in val_loader:
            data = data.to(device)
            target = target.to(device)
            output = net.forward(data)
            correct += torch.sum(
                snn.predict.getClass(output) == label
            ).data.item()
            num_samples += len(label)
            spike_loss = error.numSpikes(output, target)  # numSpikes
    print('val:', correct / num_samples)

        #writer.add_scalar("loss/test", spike_loss / len(test_loader), epoch)
        #writer.add_scalar("acc/test", correct / num_samples, epoch)

In [67]:
def _test():
    correct = 0
    num_samples = 0
    net.eval()
    with torch.no_grad():
        for data, target, label in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = net.forward(data)
            correct += torch.sum(
                snn.predict.getClass(output) == label
            ).data.item()
            num_samples += len(label)
            spike_loss = error.numSpikes(output, target)  # numSpikes
        print('test:', correct / num_samples)

        #writer.add_scalar("loss/test", spike_loss / len(test_loader), epoch)
        #writer.add_scalar("acc/test", correct / num_samples, epoch)

In [68]:
for epoch in range(1, 100 + 1):
    _train()
    if epoch % 10 == 0:
        _test()
        _val()
    if epoch % 100 == 0:
        _save_model(epoch)

Train: 0.21666666666666667
Train: 0.23333333333333334
Train: 0.4
Train: 0.4666666666666667
Train: 0.45
Train: 0.48333333333333334
Train: 0.48333333333333334
Train: 0.48333333333333334
Train: 0.48333333333333334
Train: 0.5
test: 0.55
val: 0.55
Train: 0.55
Train: 0.55
Train: 0.55
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.55
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
test: 0.55
val: 0.55
Train: 0.55
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5833333333333334
Train: 0.5666666666666667
Train: 0.5666666666666667
Train: 0.5666666666666667
test: 0.5625
val: 0.55
Train: 0.5666666666666667
Train: 0.5833333333333334
Train: 0.6
Train: 0.6
Train: 0.6
Train: 0.5833333333333334
Train: 0.6
Train: 0.6333333333333333
Train: 0.6333333333333333
Train: 0.65
test: 0.5625
val: 0.55
Train: 0.6833333333333333
Train: 0.7166666666666667
Tr

NameError: name '_save_model' is not defined